In [2]:
from azureml.core import Workspace
from azureml.core import Experiment
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from azureml.core.model import Model
import os

In [17]:
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="train_save_prediction_model_exp")
run = experiment.start_logging()

In [18]:
train, validation = train_test_split(pd.read_csv("train2.csv"),test_size=0.2,shuffle=True)

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
    
m = ['Male','male','m','M','Make','Cis Male','Man','cis male']
f = ['Female','female','F','f','Woman','woman']


def genderNormalizer(gender):
    if gender in m:
        return 'm'
    elif(gender in f):
        return 'f'
    else:
        return '-1'
    
    
train['genderNormalized'] = train.apply(lambda row: genderNormalizer(row.Gender), axis=1)
train.drop(train.loc[train['genderNormalized']=='-1'].index, inplace=True)
train = train.drop(['s.no','Timestamp','comments','Gender'],axis=1)

train_one_hot_encoded = MultiColumnLabelEncoder().fit_transform(train.drop('Age',axis=1))
train_one_hot_encoded['Age'] = train['Age']

important_feature = ['work_interfere','family_history','care_options','Age','benefits','no_employees','Country','state','leave','coworkers']

train_target = train_one_hot_encoded['treatment']
train_important_feature = train_one_hot_encoded[important_feature]

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(train_important_feature, train_target)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [23]:
model_name = "mlp_classifier"
model_file = "mlp_classifier" + ".pkl"
filename = "outputs/" + model_file
joblib.dump(value=clf, filename=filename)
run.upload_file(name=model_name, path_or_stream=filename)
run.complete()

AzureMLException: AzureMLException:
	Message: UserError: Resource Conflict: ArtifactId ExperimentRun/dcid.05e81f03-1a25-46fb-9a68-0f0dddf1abfb/mlp_classifier already exists.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "UserError: Resource Conflict: ArtifactId ExperimentRun/dcid.05e81f03-1a25-46fb-9a68-0f0dddf1abfb/mlp_classifier already exists."
    }
}

In [26]:
model = run.register_model(model_name = model_name,
                           model_path = 'mlp_classifier')

In [27]:
model = Model(ws, 'mlp_classifier')
model.download(target_dir=os.getcwd(), exist_ok=False)

'/mnt/azmnt/code/Users/live.com#ffrolicking/mlp_classifier'